In [1]:
#libraries needed for this program
from pyspark import SparkContext
from nltk.corpus import stopwords
from collections import Counter

In [2]:
#function to remove charectors from words
def charRemoveFromString(x):
	cleanString = ''.join(filter(str.isalnum, x)) 
	return cleanString

In [3]:
#function to classify catogory Wise Top Words
def catogoryWiseTopWords(all_articals_In_Catagory):
    wordsList=list()
    flag=0
    for y in all_articals_In_Catagory:
        y=y.rsplit()
        y=[charRemoveFromString(word) for word in y if word not in stopwords.words('english')]
        wordsList.append(Counter(y))
    for y in wordsList:
        if(flag):
            tempcollector=y+tempcollector
        else:
            tempcollector=y
            flag=1
    return tempcollector.most_common(10)

In [4]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")
stopWords=stopwords.words('english')
stopWords.append('')

21/10/02 02:34:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/02 02:34:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# read data from text file and split each line into words and calling catogoryWiseTopWords
nytarticals = sc.wholeTextFiles("/Users/venkatavarunnelakuditi/Desktop/input.txt").flatMap(lambda lin: lin[1].split("URL: ")).filter(lambda x: x != '').groupBy(lambda w: w[0:39]).map(lambda x : (x[0],catogoryWiseTopWords(list(x[1]))))

In [7]:
# read data from text file and split each line into words
nytarticals=nytarticals.collect()
for x in nytarticals:
    print(x)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


('http://www.nytimes.com/2016/06/30/sport', [('said', 41), ('I', 26), ('Phelps', 21), ('The', 20), ('team', 16), ('Hornacek', 15), ('', 14), ('Mets', 13), ('Knicks', 12), ('Olympic', 12)])
('http://www.nytimes.com/2016/06/30/nyreg', [('Mr', 112), ('said', 64), ('The', 37), ('students', 22), ('Pigeon', 21), ('', 20), ('50', 16), ('year', 16), ('JHS', 16), ('public', 16)])
('http://www.nytimes.com/2016/06/30/busin', [('Lionsgate', 9), ('Starz', 9), ('cable', 6), ('premium', 5), ('would', 4), ('channel', 4), ('TV', 4), ('deal', 3), ('person', 3), ('Wednesday', 3)])
('http://www.nytimes.com/2016/06/30/world', [('Islamic', 44), ('State', 39), ('Turkey', 30), ('said', 30), ('The', 25), ('fighters', 22), ('', 17), ('Syria', 16), ('Surt', 15), ('groups', 14)])
('http://www.nytimes.com/2016/06/30/us/po', [('said', 42), ('Mrs', 24), ('Mr', 20), ('Clinton', 19), ('Ms', 18), ('Democrats', 14), ('Senator', 12), ('private', 12), ('email', 12), ('Wednesday', 11)])
('http://www.nytimes.com/2016/06/30/